In [1]:
#import necessary packages
from bs4 import BeautifulSoup
from bs4 import NavigableString 
import os
from os import listdir
import shutil
from pprint import pprint
import re
import string

In [2]:
#All citation styles considered in the experiment
#american-chemical-society
#american-medical-association
# apa
# chicago-annotated-bibliography
# ieee
# modern-language-association
# national-library-of-medicine-grant-proposals
# turabian-fullnote-bibliography
# vancouver

In [3]:
#Method to check if a file exists
def deleteFile( fname ):
    if os.path.exists(fname):
        os.remove(fname)
    else:
        print("Can not delete the file as it doesn't exists")



In [81]:
#60:20:20 split for train:test:validation
#Path to the input files , after running run_commands notebook
def splitData(inputPath,documentType):
    filePath = inputPath+documentType

    #file name from the run_commands notebook output
    file = "output.no_invalid_rows.txt"
    folders=[]
    
    #Finding all styles for a particular document type
    for r, d, f in os.walk(filePath):
        for folder in d:
            folders.append(os.path.join(r, folder))
    for paths in folders:
        fname = paths+"/"+file
        
        num_lines = 0
        with open(fname, 'r') as f:
            for line in f:
                num_lines += 1
                
        #Calculate the number of lines for each split
        #60:40:40 for Train:Val:Test
        
        train_n = int(0.6*num_lines)
        test_n = int(0.2*num_lines)
        #print(train_n)
        #print(test_n)
        
        #Reading the input files linewise and append to a list
        text = open(fname, 'r')
        data = text.readlines()
        dict = []
        for line in data:
            l = line.strip().split('\n')
            dict.append(l)
        
        #Validation index
        val_n = train_n+test_n
        print(val_n)
        
        #Index splicing the data for train test and validation
        train_data= dict[0:train_n]
        val_data=dict[train_n:val_n]
        test_data = dict[val_n:]
        print("Path is " + paths)
        
        #delete if file exists
        deleteFile(paths+"/"+'train.txt')
        deleteFile(paths+"/"+'val.txt')    
        deleteFile(paths+"/"+'test.txt')
        
        #Write Train data into Disk
        file_train = open(paths+"/"+'train.txt','w')
        for element in train_data:
            file_train.write('\n'.join(element))
            file_train.write('\n')
        file_train.close()
        
        #val_data
        file_val = open(paths+"/"+'val.txt','w')
        for element in val_data:
            file_val.write('\n'.join(element))
            file_val.write('\n')
        file_val.close()
        
        #Test Data 
        file_test = open(paths+"/"+'test.txt','w')
        for element in test_data:
            file_test.write('\n'.join(element))
            file_test.write('\n')
        file_test.close()

In [93]:
#Path to the input files for all style data
inputPath= "/Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/"

#Use splitData for all the document types
splitData(inputPath,"journals")
splitData(inputPath,"proceedings")
splitData(inputPath,"abc")

3590
Path is /Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/journals/modern-language-association
3590
Path is /Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/journals/apa
3590
Path is /Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/abc/modern-language-association
Can not delete the file as it doesn't exists
Can not delete the file as it doesn't exists
Can not delete the file as it doesn't exists


In [186]:
#Combining all train,val,test files for all citation styles into one file for each document type


def combine_files(path , documentType):
    
    filePaths = path+"/"+documentType
    
    train_file_name= "train.txt"
    test_file_name="test.txt"
    val_file_name= "val.txt"
    folders=[]

    for r, d, f in os.walk(filePaths):

        for folder in d:
            folders.append(os.path.join(r, folder))
    for paths in folders:
        file= open(paths+"/"+train_file_name)
        with open(path+"/"+documentType+"_all_train.txt", "a") as f1:
            for line in file:
                #print(line)
                f1.write(line)
    for paths in folders:
        file= open(paths+"/"+ test_file_name)
        with open(path+"/"+documentType+"_all_test.txt", "a") as f1:
            for line in file:
                #print(line)
                f1.write(line)
    for paths in folders:
        file= open(paths+"/"+val_file_name)
        with open(path+"/"+documentType+"_all_val.txt", "a") as f1:
            for line in file:
                #print(line)
                f1.write(line)
        

In [198]:

#Path to all files

path = "/Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/"

#Add for different documentTypes
combine_files(path,'journals')
combine_files(path,'proceedings')
combine_files(path,'abc')

In [199]:
import time, glob
import shutil

#Combine all train test validation from all document type into one single file
inputPath= "/Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/"
trainFilename ="all_train.txt"
valFilename="all_val.txt"
testFilename="all_test.txt"
filenames = glob.glob(inputPath+'*.txt')

#TrainFile into one songle train file
with open(inputPath+trainFilename, 'wb') as outfile:
    for filename in glob.glob(inputPath+'*_train.txt'):
     
        if filename == inputPath+trainFilename:
             continue
        with open(filename, 'rb') as readfile:
            shutil.copyfileobj(readfile, outfile)

with open(inputPath+testFilename, 'wb') as outfile:
    for filename in glob.glob(inputPath+'*_test.txt'):
        if filename == inputPath+testFilename:
            continue
        with open(filename, 'rb') as readfile:
            shutil.copyfileobj(readfile, outfile)
            
with open(inputPath+valFilename, 'wb') as outfile:
    for filename in glob.glob(inputPath+'*_val.txt'):
        if filename == inputPath+valFilename:
             continue
        with open(filename, 'rb') as readfile:
            shutil.copyfileobj(readfile, outfile)

In [200]:
#Shuffle the train/test/val for each type 
import random
def shuffle_data(path,inputFilename,outputFilename):
    lines = open(path+inputFilename).readlines()
    random.shuffle(lines)
    open(path+outputFilename, 'w').writelines(lines)



In [201]:
path="/Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/"
trainFile="all_train.txt"
trainshuffle="train.txt"
valFile="all_val.txt"
valShuffle="val.txt"
testFile="all_test.txt"
testShuffle="test.txt"

# Shuffle train, test and valiation data
shuffle_data(path,trainFile,trainshuffle)
shuffle_data(path,valFile,valShuffle)
shuffle_data(path,testFile,testShuffle)

In [211]:
#To divide the entire test/train/val into number of folds
def create_folds(path,fileType):
    fname = path+fileType+".txt"
    num_lines = 0

    with open(fname, 'r') as f:
        for line in f:
            num_lines += 1
    print(num_lines)
    window = int(0.1*num_lines)
    print(window)
    #print(each_fold)


    text = open(fname, 'r')
    data = text.readlines()
    dict = []
    for line in data:
        l = line.strip().split('\n')
        dict.append(l)
    path_to_folds = '/Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/folds/'
    start_range = 0
    temp_range = start_range+window
    end_range=len(dict)
    for i in range(11):
        #fold_i = dict[start_range:end_range]
        file = open(path_to_folds+fileType+str(i)+'.txt','w')
        
       
        if not (temp_range > end_range):
            #print(file)
            print(str(start_range)+":"+ str(temp_range))
            fold = dict[start_range:temp_range]
            for element in fold:
                print(element)
                file.write('\n'.join(element))
                file.write('\n')  
            start_range = start_range + window
            temp_range = start_range + window
        else:
            print("here")
            #print(str(temp_range-window) +":"+str(end_range))
            last_fold = dict[temp_range-window:end_range]
            print(str(temp_range-window)+":"+str(end_range))
            #print("last"+ str(file))

            for last_elem in last_fold:
                file.write('\n'.join(last_elem))
                file.write('\n')     
        file.close()



In [237]:
path="/Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/"

#Note folders to be created manually
!cd styleData/
!pwd
# create_folds(path,'test')
# create_folds(path,'train')
# create_folds(path,'val')

/Users/bipashabanerjee/Documents/CS/sem3/publish


In [ ]:
#Concatenate all train and val in folds which is done manually.
#the file structure should look like folds/training/{0...9}/{train/val}_{0-9}.txt

In [214]:
def perform_function(inputFile, outputFile):

    #write to file the output after stripping the punctuations
    #inputFile = "/Users/bipashabanerjee/Documents/CS/sem3/project/styleData_old/report/modern-language-association/check.txt"
    punctuation = string.punctuation 
    #print(punctuation)
    text = open(inputFile, 'r')
    data = text.readlines()
    dict = []
    for line in data:
        l = line.strip().split('\n')
        dict.append(l)
    #print(dict)
    str1 = str(dict[0])

    soup = BeautifulSoup(str1)


    tag_list= []
    for tag in soup.find_all():
        tag_list.append(tag.name)

    tag_list.remove('html')
    tag_list.remove('body')
    tag_list.remove('p')
    #To find the tags associated with the particular citation style


    for m in dict:
        for text in m:

            soup1 = BeautifulSoup(text)
            body = soup1.find('body')
            newtag_list= []
            for newtag in body.find_all():
                newtag_list.append(newtag.name)
    #         for tag in newtag_list:
    #             if(newtag_list=='p'):
    #                 newtag_list.remove('p')

            for text_b in body:
                file = open(outputFile,"a+")
                if isinstance(text_b, NavigableString) == True:
                    token = text_b.split()
                    for t in token:
                        file.write(t + "\t"+ "extra"+'\n')

                elif text_b.name =='p':


                    newtag_list.remove('p')

                    for tag in newtag_list:
                        text = soup.find_all(tag)
                        for tag_text in text:

                            split_text=re.split('\s+', tag_text.text )
                            for t in split_text:
                                file.write(t.replace(',','') + "\t"+ tag)
    #                         for tag in newtag_list:

    #                             text = soup.find_all(tag)
    #                             for tag_text in text:


                else:

                    split_text=re.split('\s+', text_b.text )   

                    for t in split_text:

                        if text_b.name == "author":

                            if(t in punctuation):
                                file.write(t+"\t"+'punctuation'+'\n')
                            else:

                                file.write(t.replace(',','')+"\t"+text_b.name+'\n')




                        else:
                            if(t!='' and t ):
                                file.write(t+"\t"+ text_b.name+'\n')

            file.write("\n")



In [225]:
#To transform data into token,label format for each train,val and test for each fold
#Change the path to test/train/val accordingly in the input and output path mentioned below
for i in range(10):
    inputFile = "/Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/folds/"+str(i)+"/train"+str(i)+".txt"
    outputFile = "/Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/training/folds/"+str(i)+"/train.txt"

    perform_function(inputFile,outputFile)

In [226]:
#combine all tests from all folds

all_test_files = "/Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/training/folds/"
outputPath = "/Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/test/"

test_file_name="test.txt"

folders=[]

for r, d, f in os.walk(all_test_files):
    
    for folder in d:
        folders.append(os.path.join(r, folder))
for paths in folders:
    file= open(paths+"/"+test_file_name)
    with open(outputPath+"test.txt", "a") as f1:
        for line in file:
            #print(line)
            f1.write(line)

In [1]:
#remove excess test data from each folds
test_file_name="test.txt"
folders=[]
for r, d, f in os.walk(all_test_files):
    
    for folder in d:
        folders.append(os.path.join(r, folder))
for paths in folders:
    os.remove(paths+"/test.txt")
    #file= open(paths+"/train.txt")

In [219]:

#Utlity function to calculate number of lines
fname = "/Users/bipashabanerjee/Documents/CS/sem3/publish/styleData/train.txt"
# fname="/Users/bipashabanerjee/Documents/CS/sem3/project/styleData/journals/apa/train.txt"
num_lines=0
with open(fname, 'r') as f:
    for line in f:
        num_lines += 1
        
print(num_lines)





8079
